In [ ]:
import os, sys
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil
import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
pygeo = geo2d.PyDraw()
from ROOT import larocv
from ROOT import std
from ROOT import cv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
%matplotlib inline
import numpy as np
import math

In [ ]:
proc = larcv.ProcessDriver('ProcessDriver')
# CFG="../cfg/prod_fullchain_ssnet_combined_test.cfg"
# CFG="../cfg/prod_fullchain_ssnet_combined.cfg"

CFG="../jarret/prod_fullchain_truth_combined_numu.cfg"

preprocessed=False

print "Loading config... ",CFG
proc.configure(CFG)
flist=ROOT.std.vector('std::string')()
# flist.push_back("/Users/dayajun/Desktop/for_rui/intrinsic_nue/out_pyroi/larcv_fcn_out_0000_0099.root")
flist.push_back("../jarret/NuMuGolden1_larcv.root")
proc.override_input_file(flist)
proc.override_output_file("/tmp/cacca.root")
proc.override_ana_file("/tmp/test.root")

# vinroi_id  = proc.process_id("VertexInROI")
reco_id    = proc.process_id("LArbysImage")
larbysimg  = proc.process_ptr(reco_id)

proc.initialize()

In [ ]:
event=1
proc.batch_process(event,1)

In [ ]:
mgr=larbysimg.Manager()
# help(mgr)
# for shit in mgr.Names():
#     print shit


In [ ]:
img_v  = [None,None,None]
oimg_v = [None,None,None]
track_img_v  = [None,None,None]
otrack_img_v = [None,None,None]
shower_img_v  = [None,None,None]
oshower_img_v = [None,None,None]
for ix in xrange(3):
    oimg_v[ix]        = pygeo.image(mgr.OriginalInputImages(0)[ix])
    otrack_img_v[ix]  = pygeo.image(mgr.OriginalInputImages(1)[ix])
#     oshower_img_v[ix] = pygeo.image(mgr.OriginalInputImages(2)[ix])
    img_v[ix]         = pygeo.image(mgr.InputImages(0)[ix])
    track_img_v[ix]   = pygeo.image(mgr.InputImages(1)[ix])
#     shower_img_v[ix]  = pygeo.image(mgr.InputImages(2)[ix])

### SSNet Output

In [ ]:
for plane in xrange(len(track_img_v)):
    oshower_img = np.where(oshower_img_v[plane]>10.0,85.0 ,0.0).astype(np.uint8)
    otrack_img  = np.where(otrack_img_v[plane] >10.0,160.0,0.0).astype(np.uint8)
    shower_img  = np.where(shower_img_v[plane] >10.0,85.0 ,0.0).astype(np.uint8)
    track_img   = np.where(track_img_v[plane]  >10.0,160.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(10,10))
    oimg = oshower_img + otrack_img
    img  = shower_img  + track_img
    ax.set_xlabel('Time [6 ticks]',fontsize=20)
    ax.set_ylabel('Wire',fontsize=20)
    ax.imshow(img,cmap='jet',interpolation='none',vmin=0.,vmax=255.)
    ax.tick_params(labelsize=20)
    plt.tight_layout()
    plt.show()

### Track Vertex Seeds

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trackvertexseeds")
print "Algorithm ID:",algoid
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img1 = track_img_v[plane]
    shape_img2 = shower_img_v[plane]
    shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
    shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
    shape_img=shape_img1+shape_img2
    ax.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ix=0

    ####################################
    vertex_seeds_v = dm.Data(1,plane).as_vector()
    cluscomp_v     = dm.Data(1,3+plane).as_vector()

    #plot the atomics
    for cluscomp_id in xrange(cluscomp_v.size()):
        cluscomp = cluscomp_v[cluscomp_id]
        for atomic_id in xrange(cluscomp.size()):
            atomic = cluscomp[atomic_id]
            pts=[[atomic[p_id].x,atomic[p_id].y] for p_id in xrange(atomic.size())]
            if len(pts)==0: continue
            pts.append(pts[0])
            pts=np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-',lw=3,color=colors[ix],alpha = 0.8)
            ix+=1

    pts_v = np.array([[vertex_seeds_v[i].x,
                       vertex_seeds_v[i].y] for i in xrange(vertex_seeds_v.size())])
    if pts_v.size>0:
        plt.plot(pts_v[:,0],pts_v[:,1],'*',markersize=20,color='yellow')

    ####################################
    plt.tight_layout()
    try:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    except ValueError:
        pass

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    ax.grid()
    plt.show()

### Track Vertex Estimate

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trackvertexestimate")
print "Algorithm ID:",algoid
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img1 = track_img_v[plane]
    shape_img2 = shower_img_v[plane]
    shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
    shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
    shape_img=shape_img1+shape_img2
    ax.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)
    ix=0
    
    ####################################
    vertex_vv  = dm.Data(algoid,0).as_vector()
    cluscomp_v = dm.Data(algoid-1,3+plane).as_vector()
    for cluscomp_id in xrange(cluscomp_v.size()):
        cluscomp = cluscomp_v[cluscomp_id]
        for atomic_id in xrange(cluscomp.size()):
            atomic = cluscomp[atomic_id]
            pts=[[atomic[p_id].x,atomic[p_id].y] for p_id in xrange(atomic.size())]
            if len(pts)==0: continue
            pts.append(pts[0])
            pts=np.array(pts)
            plt.plot(pts[:,0],pts[:,1],'-o',lw=3,color=colors[ix], alpha = 0.5)
            ix+=1

    pts_v = np.array([[vertex_vv[i].cvtx2d_v[plane].center.x,
                       vertex_vv[i].cvtx2d_v[plane].center.y] for i in xrange(vertex_vv.size())])
    if pts_v.size>0:
        print "\t<=== Track Vertex Candidates @ ===>\n",pts_v
        ax.plot(pts_v[:,0],pts_v[:,1],'*',markersize=30,color='cyan')
    ####################################
    
    plt.tight_layout()
    try:
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    except ValueError:
        pass

    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    ax.grid()
    plt.show()

### Angular Metric

In [ ]:
algoid=dm.ID("trackvertexestimate")
alg=mgr.GetClusterAlg(algoid).Algo()

tickscore0_y=[]
tickscore0_x=[]

score0_v = alg.TimeBinnedScore0Mean()
for idx in xrange(score0_v.size()):
    v = score0_v[idx]
    tickscore0_y.append(v)
    tickscore0_x.append(idx*1 + alg.TimeBinMin())

tickscore1_y=[]
tickscore1_x=[]
score1_v = alg.TimeBinnedScore1Mean()
for idx in xrange(score1_v.size()):
    v = score1_v[idx]
    tickscore1_y.append(v)
    tickscore1_x.append(idx*1 + alg.TimeBinMin())

tickscore0_x = np.array(tickscore0_x)
tickscore0_y = np.array(tickscore0_y)
tickscore1_x = np.array(tickscore1_x)
tickscore1_y = np.array(tickscore1_y)
ymin = tickscore0_y.min()
ymax = tickscore0_y.max()
if ymin > tickscore1_y.min(): ymin = tickscore1_y.min()
if ymax < tickscore1_y.max(): ymax = tickscore1_y.max()

fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
ax.plot(tickscore0_x,tickscore0_y,marker='o',linestyle='-',color='red',markersize=10)
ax.plot(tickscore1_x,tickscore1_y,marker='o',linestyle='--',
         markeredgewidth=1,markeredgecolor='blue',markerfacecolor='None',markersize=10)


minimum_v  = alg.TimeBinnedScoreMinIndex()
minrange_v = alg.TimeBinnedScoreMinRange()
for idx in xrange(minimum_v.size()):
    xval = tickscore0_x[minimum_v[idx]]
    ax.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
    xstart = tickscore0_x[minrange_v[idx].first]
    xend   = tickscore0_x[minrange_v[idx].second]
    ax.axvspan(xstart,xend, alpha=0.3, color='orange')

ax.set_xlabel('Time [6 ticks]',fontsize=20,fontweight='bold')
ax.set_ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
ax.tick_params(labelsize=20)
plt.tight_layout()
ax.set_ylim(-1,ymax*1.1)
ax.set_xlim(tickscore0_x.min(),tickscore0_x.max())
ax.grid()
plt.show()


### Track Type Particle Clustering

In [ ]:
assman=dm.AssManager()
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10
dm=mgr.DataManager()
algoid=dm.ID("trk_vertexparticleclustermaker")
vtx_data=dm.Data(algoid-2,0).as_vector()

####################################
for ix,vtx in enumerate(vtx_data):
    
    print "<==========================================================================>"
    print "<===================Start Vertex3D number ",ix," ==========================>"
    cvtx_v = vtx.cvtx2d_v
    
    for plane in xrange(3):
        cvtx = cvtx_v[plane]
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        par_data=dm.Data(algoid,plane)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0:continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=4)

        ax.plot(cvtx.center.x,
                cvtx.center.y,
                'o',color='red',markersize=10)
        
        circle_=patches.Circle((cvtx.center.x,cvtx.center.y),
                                cvtx.radius,fc='none',ec='cyan',lw=5,alpha=0.5)
        
        print "Vertex",ix,"plane @",plane,"..."
        for xs in cvtx.xs_v:
            print  "xs @ [",xs.pt.x,",",xs.pt.y,"]"
            ax.plot(xs.pt.x,xs.pt.y,'o',color='orange',markersize=10,alpha=0.7)
            
        ax.add_patch(circle_)
        ax.set_aspect(1.0)
        plt.tight_layout()
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.tick_params(labelsize=20)
        plt.show()
    print "<===================End   Vertex3D number ",ix," ==========================>"
    print "<==========================================================================>"
####################################

### Linear Track Cluster

In [ ]:
dm=mgr.DataManager()
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10

dm=mgr.DataManager()
algoid=dm.ID("lineartrackfinder")
data=dm.Data(algoid,0)
lintrk_v=data.as_vector()

print "Found...",lintrk_v.size(),"...linear track clusters"

for strack_n,strack in enumerate(lintrk_v):
    print "<===================================================================================>"
    print "<===================Start LinearTrack number ",strack_n," ==========================>"

    # the only good one...
    
    e13d=strack.edge1
    e23d=strack.edge2
    print "Edges ",e13d,e23d
    for plane in xrange(3):
        strack2d = strack.get_cluster(plane)
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        plt.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)
        if strack2d.ctor.size()>0:
            ctor = [[pt.x,pt.y] for pt in strack2d.ctor]
            ctor.append(ctor[0])
            ctor=np.array(ctor)

            ax.plot(ctor[:,0],ctor[:,1],'-o',lw=3)

            e1=strack2d.edge1
            e2=strack2d.edge2

            ax.plot(e1.x,e1.y,'*',color='orange',markersize=20)
            ax.plot(e2.x,e2.y,'*',color='yellow',markersize=20)

            try:
                vtx2d=e13d.vtx2d_v[plane]
                pt=vtx2d.pt
                ax.plot(pt.x,pt.y,'o',color='green',markersize=40,alpha=0.5)
            except:
                pass

            try:
                vtx2d=e23d.vtx2d_v[plane]
                pt=vtx2d.pt
                ax.plot(pt.x,pt.y,'o',color='green',markersize=40,alpha=0.5)
            except:
                pass

        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.set_aspect(0.8)
        ax.grid()
        plt.show()
        
    print "<===================End   LinearTrack number ",strack_n," ==========================>"
    print "<===================================================================================>"


In [ ]:
dm=mgr.DataManager()
algoid=dm.ID("showervertexestimate")
data=dm.Data(algoid,0)
print "Got ",data.as_vector().size()," shower 3D vertex estimate"
for vtxid,vtx3d in enumerate(data.as_vector()):
    print "<=============================================================================>"
    print "<===================Start concrete vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels = np.where(shape_img>0.0)
        
        cvtx=vtx3d.cvtx2d_v[plane]
        ax.plot(cvtx.center.x,cvtx.center.y,'o',color='red',markersize=10)
        circle_=matplotlib.patches.Circle((cvtx.center.x,cvtx.center.y),
                                          cvtx.radius,fc='none',ec='cyan',lw=5)
        for xs in cvtx.xs_v:
            ax.plot(xs.pt.x,xs.pt.y,'o',color='orange',markersize=10)

        ax.add_patch(circle_)
        ax.set_aspect(1.0)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_xlabel('Time [6 ticks]',fontsize=20)
        ax.set_ylabel('Wire',fontsize=20)
        ax.set_title("Vertex Type = %d"%vtx3d.type,fontsize=30)
        ax.set_aspect(0.8)
        plt.show()
    print "<===================end concrete vertex ",vtxid," ============================>"
    print "<=============================================================================>"

### Shower + Track Particle Clusters

In [ ]:
#New VertexCluster
algoid=dm.ID("showervertexestimate")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

for vtxid,vtx in enumerate(vtx_data):
    vtxid+=1
    for plane in xrange(3):
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        algoid=dm.ID("shower_vertexparticleclustermaker1")
        par_data=dm.Data(algoid,plane)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)
        
        
        algoid=dm.ID("shower_vertexparticleclustermaker2")
        par_data=dm.Data(algoid,plane)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        plt.show()


### Combined Vertices

In [ ]:
#New VertexCluster
# algoid=dm.ID("combinedvertexanalysis")
assman=dm.AssManager()
algoid=dm.ID("combinevertex")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

for vtxid,vtx in enumerate(vtx_data):
    print "<=============================================================================>"
    print "<===================Start combined vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        par_data=dm.Data(algoid,plane+1)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        ax.plot(vtx.vtx2d_v[plane].pt.x,vtx.vtx2d_v[plane].pt.y,'*',color='cyan',markersize=20)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_title("Vertex Type: %d"%vtx.type)
#         ax.set_ylim(1200,1500)
#         ax.set_xlim(700,900)
        print "<=============================================================================>"
        print "<===================End combined vertex ",vtxid," ==========================>"
        plt.show()


In [ ]:
assman=dm.AssManager()
algoid=dm.ID("matchoverlap")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

for vtxid,vtx in enumerate(vtx_data):
    print "<=============================================================================>"
    print "<===================Start combined vertex ",vtxid," ==========================>"
    for plane in xrange(3):
        fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
        shape_img1 = track_img_v[plane]
        shape_img2 = shower_img_v[plane]
        shape_img1=np.where(shape_img1>10.0,255.0,0.0).astype(np.uint8)
        shape_img2=np.where(shape_img2>10.0,100.0,0.0).astype(np.uint8)
        shape_img=shape_img1+shape_img2
        ax.imshow(shape_img,cmap='Greys',interpolation='none')
        nz_pixels=np.where(shape_img>0.0)

        par_data=dm.Data(algoid,plane+1)

        ass_t = np.array(assman.GetManyAss(vtx,par_data.ID()))
        if ass_t.size==0:continue

        par_data_v=par_data.as_vector()
        
        for id_ in ass_t:
            ctor=[[pt.x,pt.y] for pt in par_data_v[id_]._ctor]
            if len(ctor)==0: continue
            ctor.append(ctor[0])
            ctor=np.array(ctor)
            ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)

        ax.plot(vtx.vtx2d_v[plane].pt.x,vtx.vtx2d_v[plane].pt.y,'*',color='cyan',markersize=20)
        ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
        ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
        ax.set_title("Vertex Type: %d"%vtx.type)
#         ax.set_ylim(1200,1500)
#         ax.set_xlim(700,900)
        print "<=============================================================================>"
        print "<===================End combined vertex ",vtxid," ==========================>"
        plt.show()


# dQ/dx

In [ ]:
algoid=dm.ID("combinedvertexanalysis")
data=dm.Data(algoid,0)
vtx_data=dm.Data(algoid,0).as_vector()

In [ ]:
larocv.AlgoFunctionLoader()

planev = [0,1,2]
vtx3d_idx = 0
print "<=========================Vertex ",vtx3d_idx,"=========================>"
for plane in planev:
    mask = []
    mask_img = []
    print "<======================Plane ",plane,"======================>"
    fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
    vcir = vtx_data[vtx3d_idx].cvtx2d_v[plane]
    cir  = geo2d.Circle("float")(vcir.center.x, vcir.center.y, vcir.radius)
    mask = larocv.MaskImage(mgr.InputImages(0)[plane], cir, 0 , False)
    mask_img = pygeo.image(larocv.MaskImage(mgr.InputImages(0)[plane], cir, 0 , False))
    mask_img = np.where(mask_img>10.0,255.0,0.0).astype(np.uint8)
    ax.imshow(mask_img,cmap='Greys',interpolation='none')
    if larocv.FindNonZero(mask).size() < 2 : 
        plt.show()
        continue

    ctor_v = larocv.FindContours(mask)
    for ctor in ctor_v:
        ctors=[[pt.x,pt.y] for pt in ctor]
        if len(ctors)==0: continue
        ctors.append(ctors[0])
    ctor=np.array(ctors)
    ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)
    ax.plot(vcir.center.x, vcir.center.y,'*', color = 'red', markersize = 5)
    plt.show()

In [ ]:
def findxs(pt,pca):
    dir = pca.dir
    pt0 = pca.pt
    k = dir.y/dir.x
    x0=pt0.x
    y0=pt0.y
    if k >= -1 and k<=1:
        pt.x = pt.x
        pt.y = k*pt.x-k*x0+y0
        return pt
    else:
        pt.x = (pt.y-y0 +k*x0)/k
        pt.y = pt.y
        return pt

def findpt0(pt,pca):
    dir = pca.dir
    k = dir.y/dir.x
    if k >=-1 and k <=1:
        pt.y = pca.y(pt.x)
    else:
        pt.x = pca.x(pt.y)
    return pt

In [ ]:
planev = [0,1,2]
vtx3d_idx = 1
print "<=========================Vertex ",vtx3d_idx,"=========================>"
for plane in planev:
    print "<======================Plane ",plane,"======================>"
    mask = []
    mask_img = []
    fig,ax = plt.subplots(figsize=(12,12),facecolor='w')
    vcir = vtx_data[vtx3d_idx].cvtx2d_v[plane]
    cir  = geo2d.Circle("float")(vcir.center.x, vcir.center.y, vcir.radius)
    mask = larocv.MaskImage(mgr.InputImages(0)[plane], cir, 0 , False)
    mask_img = pygeo.image(mask)    
    mask_img = np.where(mask_img>10.0,255.0,0.0).astype(np.uint8)
    ax.imshow(mask_img,cmap='Greys',interpolation='none')
    
    if larocv.FindNonZero(mask).size() <2 : continue
    
    ctor_v = larocv.FindContours(mask)
    for ctor in ctor_v:
        ctors=[[pt.x,pt.y] for pt in ctor]
        if len(ctors)==0: continue
        ctors.append(ctors[0])
    ctor=np.array(ctors)
    ax.plot(ctor[:,0],ctor[:,1],'-',lw=2,alpha=0.8)
    ax.plot(vcir.center.x, vcir.center.y,'*', color = 'red', markersize = 5)
    plt.show()

    if larocv.FindNonZero(mask).size() <2 : continue
    pca  = larocv.CalcPCA(mask)
    
    ptn  = 0
    pt0  = (pca.pt.x, pca.pt.y)
    dist_v = []
    dq_v = []
    for row in xrange(mask_img.shape[0]):
        for col in xrange(mask_img.shape[1]):
            if mask_img[row,col]>0: 
                pt.y = row
                pt.x = col
                pt00 = findpt0(pt,pca)
                break
    pt0 = (pt00.x, pt00.y)

    for row in xrange(mask_img.shape[0]):
        for col in xrange(mask_img.shape[1]):
            if mask_img[row,col]>0: 
                ptn+=1
                pt.y = row
                pt.x = col
                spt = (findxs(pt,pca).x, findxs(pt,pca).y)
                dist = np.sqrt(np.power(spt[0]-pt0[0],2)+np.power(spt[1]-pt0[1],2))
                dist_v.append(dist)
                dq_v.append(mask_img[row][col])

    vtx_dist = np.sqrt(np.power(vcir.center.x-pt0[0],2)+np.power(vcir.center.y-pt0[1],2))
    
    if len(dist_v) != len(dq_v): 
        continue

    bins = 40
    bin_diff = max(dist_v)- min(dist_v)
    binsize = bin_diff/bins
    
    dqdx_v = [0]*(bins+1)
    dqdx_left = []
    dqdx_right = []
    for x in xrange(len(dist_v)): 
        idx = int(dist_v[x]/binsize)
        dqdx_v[idx]+= dq_v[x]
        
    dqdx_v = np.array(dqdx_v).astype(np.double)
    mask = np.isfinite(dqdx_v)
    for idx in xrange (len(dqdx_v)):
        if dqdx_v[idx] == 0:
            mask[idx] = False
    plt.plot(np.arange(bins+1)[mask],dqdx_v[mask], '-o')
    plt.axvline(x=vtx_dist/binsize, color='cyan', linestyle='--')

    plt.grid()
    plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(12,12),facecolor='w')

plt.imshow(pygeo.image(mask),cmap='Greys',interpolation='none')
print np.where(pygeo.image(mask).flatten()>0)[0].size

line=larocv.CalcPCA(mask)
yy=np.array([line.y(0),line.y(80)])
xx=np.array([0,80])
ax=plt.gca()
# ax.set_xlim(0,100)
# ax.set_ylim(0,50)
ax.grid()
ax.plot(xx,yy,'-o',color='red',markersize=10)
plt.show()